In [1]:
import sys
import datetime as dt

import tensorflow as tf
import pandas as pd
import numpy as np

from pathlib import Path

from src import paths
from src.datasets import daocensus_text
from src.model_selection import timeFreqSplitCurrent

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))
print("Numpy version: {}".format(np.__version__))

System version: 3.9.18 (main, Oct 24 2023, 09:18:18) 
[GCC 11.4.0]
Tensorflow version: 2.16.1
Numpy version: 1.26.4


/home/daviddavo/upm-tfm-notebooks/src/model_selection.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


### Parameters

In [2]:
ORG_NAME = 'Decentraland'
ORG_PLATFORM: str = 'snapshot'
USE_ORG_NAMES: bool = True

# Evaluation
K_RECOMMENDATIONS: list[int] = [1,3,5,10,100]
SPLITS_FREQ: str = 'W-THU' # split weekly
SPLITS_NORMALIZE = True # Wether or not to move everything to 00:00
LAST_SPLITS = 10 # Use just last 10 splits
CUTOFF_DATE_STR: str = '2023-07-29'

MDF_FILE: str = None

BATCH_SIZE: int = 64
EPOCHS: int = 10
RANDOM_SEED: int = 42

In [3]:
CUTOFF_DATE = dt.datetime.fromisoformat(CUTOFF_DATE_STR) if CUTOFF_DATE_STR else None

print('CUTOFF_DATE', CUTOFF_DATE, type(CUTOFF_DATE))
# Metrics Dataframe
MDF_FILE = Path(MDF_FILE) if MDF_FILE else paths.pln_mdf(ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, cutoff_date=CUTOFF_DATE)
print(MDF_FILE)

CUTOFF_DATE 2023-07-29 00:00:00 <class 'datetime.datetime'>
data/pln/mdf_Decentraland_W-THU_normalize-cutoff_date=2023-07-29 00:00:00.pkl


### Load dataset

> xDeepFM uses the FFM format as data input: `<label> <field_id>:<feature_id>:<feature_value>`
>
> Each line represents an instance, `<label>` is a binary value with 1 meaning positive instance and 0 meaning negative instance. Features are divided into fields. For example, user's gender is a field, it contains three possible values, i.e. male, female and unknown. Occupation can be another field, which contains many more possible values than the gender field. Both field index and feature index are starting from 1.

```
The data format of LIBFFM is:

<label> <field1>:<feature1>:<value1> <field2>:<feature2>:<value2> ...
.
.
.

`field' and `feature' should be non-negative integers. See an example `bigdata.tr.txt.'

It is important to understand the difference between `field' and `feature'. For example, if we have a raw data like this:

Click  Advertiser  Publisher
=====  ==========  =========
    0        Nike        CNN
    1        ESPN        BBC

Here, we have 

    * 2 fields: Advertiser and Publisher

    * 4 features: Advertiser-Nike, Advertiser-ESPN, Publisher-CNN, Publisher-BBC

Usually you will need to build two dictionares, one for field and one for features, like this:
    
    DictField[Advertiser] -> 0
    DictField[Publisher]  -> 1
    
    DictFeature[Advertiser-Nike] -> 0
    DictFeature[Publisher-CNN]   -> 1
    DictFeature[Advertiser-ESPN] -> 2
    DictFeature[Publisher-BBC]   -> 3

Then, you can generate FFM format data:

    0 0:0:1 1:1:1
    1 0:2:1 1:3:1

Note that because these features are categorical, the values here are all ones.
```

In [4]:
print(ORG_NAME, ORG_PLATFORM, USE_ORG_NAMES, CUTOFF_DATE)
dfv, dfp = daocensus_text.get("./data/daos-census-text", ORG_NAME, ORG_PLATFORM, use_org_names=USE_ORG_NAMES, cutoff_date=CUTOFF_DATE)
dfv[['voter', 'proposal']] = dfv[['voter', 'proposal']].astype(str)
dfp[['id']] = dfp[['id']].astype(str)
dfp = dfp.set_index('id')
print(dfv.info())
print(dfp.info())

Decentraland snapshot True 2023-07-29 00:00:00
Warning, removing 116560 votes without proposal
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116560 entries, 0 to 116559
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   platform       116560 non-null  object        
 1   name           116560 non-null  object        
 2   id             116560 non-null  object        
 3   proposal       116560 non-null  object        
 4   deployment     116560 non-null  object        
 5   platform_vote  116560 non-null  object        
 6   voter          116560 non-null  object        
 7   date           116560 non-null  datetime64[us]
 8   choice         116560 non-null  object        
 9   weight         116560 non-null  float64       
dtypes: datetime64[us](1), float64(1), object(8)
memory usage: 8.9+ MB
None
<class 'pandas.core.frame.DataFrame'>
Index: 1942 entries, a76b1c38-ac9d-5153-9003-8b3ade553c5

In [5]:
from recommenders.datasets import pandas_df_utils as pdu

In [6]:
dfpv = dfv.merge(dfp, how='inner', left_on=['platform', 'proposal'], right_on=['platform', 'id'], suffixes=('', '_p')).drop(columns=['name_p', 'deployment_p', 'votes_count'])
dfpv['author'] = dfpv['author'].astype(str)
dfpv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116560 entries, 0 to 116559
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   platform             116560 non-null  object        
 1   name                 116560 non-null  object        
 2   id                   116560 non-null  object        
 3   proposal             116560 non-null  object        
 4   deployment           116560 non-null  object        
 5   platform_vote        116560 non-null  object        
 6   voter                116560 non-null  object        
 7   date                 116560 non-null  datetime64[us]
 8   choice               116560 non-null  object        
 9   weight               116560 non-null  float64       
 10  platform_deployment  116560 non-null  object        
 11  platform_proposal    116560 non-null  object        
 12  author               116560 non-null  object        
 13  date_p        

In [7]:
c = pdu.LibffmConverter()
# c.fit_transform(dfpv[['voter', 'proposal', 'weight', 'author']].assign(rating=1))
c.fit_transform(dfpv[['voter', 'proposal']].assign(rating=1))

,rating,voter,proposal
0,1,1:1:1,2:7269:1
1,1,1:2:1,2:7269:1
2,1,1:3:1,2:7269:1
3,1,1:4:1,2:7269:1
4,1,1:5:1,2:7269:1
...,...,...,...
116555,1,1:2120:1,2:9210:1
116556,1,1:307:1,2:9210:1
116557,1,1:3363:1,2:9210:1
116558,1,1:1284:1,2:9210:1


In [8]:
all_folds = list(timeFreqSplitCurrent(
    dfpv, 
    SPLITS_FREQ, 
    dfp, 
    return_open=True,
    user_col='voter',
    item_col='proposal',
    timestamp_col='date',
    remove_not_in_train_col='voter'
))
folds = all_folds[-LAST_SPLITS:]

## Model

In [9]:
from recommenders.models.deeprec.models.xDeepFM import XDeepFMModel
from recommenders.models.deeprec.io import iterator
from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources, prepare_hparams

In [10]:
import os

data_path = Path('./data/xdeepfm')

yaml_file = data_path / 'xDeepFM.yaml'
if not yaml_file.exists():
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/deeprec/', data_path, 'xdeepfmresources.zip')

In [31]:
hparams = prepare_hparams(
    # yaml_file=yaml_file,
    model_type='xDeepFM',
    data_format='ffm',
    # FEATURE_COUNT=2300000, 
    # FIELD_COUNT=3, 
    MODEL_DIR='/tmp/xdeepfm.model',
    FEATURE_COUNT=c.feature_count,
    FIELD_COUNT=c.field_count,
    dim=10,
    activation=['relu', 'relu'],
    method='classification',
    cross_l2=0.01, 
    embed_l2=0.01, 
    layer_l2=0.01,
    learning_rate=0.002, 
    batch_size=BATCH_SIZE, 
    epochs=EPOCHS, 
    cross_layer_sizes=[20, 10], 
    init_value=0.1, 
    layer_sizes=[20,20],
    use_Linear_part=True, 
    use_CIN_part=True, 
    use_DNN_part=True,
    loss='log_loss',
)

In [32]:
model = XDeepFMModel(hparams, iterator.FFMTextIterator, seed=RANDOM_SEED)

Add linear part.
Add CIN part.
Add DNN part.


In [33]:
!wc -l /tmp/train.ffm
!wc -l /tmp/test.ffm

212258 /tmp/train.ffm
354 /tmp/test.ffm


In [34]:
pdu.negative_feedback_sampler(folds[0].train, col_user='voter', col_item='proposal', col_feedback='rating')

,platform,name,id,proposal,deployment,platform_vote,voter,date,choice,weight,platform_deployment,platform_proposal,author,date_p,title,description,start,end,rating
0,snapshot,Decentraland,4ebd9e1f-a5be-5fb3-be0a-89ff51c91f90,695d87ce-7de4-5042-82d3-0870597845a7,41fd8de5-f8e2-5023-86a3-825c49e9ad7f,QmXiEBWSikJ7jtkGfJZK5v7bwrxwvfF7rb9SFGpKQ6noRk,0x0019220ddabe5b8438a6be6ebca31ef01ca69965,2022-02-16 21:20:39,"[0, 0, 0, 262.648291025732, 100, 0]",362.0,snapshot.dcl.eth,QmdxCQCREhEV6yQvLnqjsp3tVcMfXX4ZqUrdwQBvt8GVVT,0x5e23d08324f017d5425e59a2782c9ae27ace0958,2022-02-14 23:03:47,WeMeta Builder Tag,> by 0x8ac69421a24690d62d7eda58e0daaa21a249d98...,2022-02-14 23:03:00,2022-02-28 23:03:00,1
1,snapshot,Decentraland,9bde9716-cdcf-545b-a10f-2d9ed090f1e8,f55d6c6a-dc72-521e-8d06-864c272ca6ca,41fd8de5-f8e2-5023-86a3-825c49e9ad7f,QmTAMckAD4j8oEKHeEMjbaojMe58XJGYKzMDQGgaTX1E3S,0x0019220ddabe5b8438a6be6ebca31ef01ca69965,2021-11-02 08:51:51,"[0, 0, 0, 100, 0, 0]",100.0,snapshot.dcl.eth,Qmf4SKp15xZhXmA2LhgY89jbiv2SH695nT1wxUr6c5uWbV,0x5e23d08324f017d5425e59a2782c9ae27ace0958,2021-10-29 23:46:28,"Add the location 83,70 to the Points of Interest",> by 0x509cdebba232e7b7f603db5c93d1e4f1fed9c6f...,2021-10-29 23:46:00,2021-11-05 23:46:00,1
2,snapshot,Decentraland,6b347552-a325-545a-8831-3c163caabe9b,719d92df-a661-59a4-a364-51148c6b336c,41fd8de5-f8e2-5023-86a3-825c49e9ad7f,QmWnDutv2i6Kze7Rqz1PyEybtUXNL1xzU8tBVuLoPnusgi,0x0019220ddabe5b8438a6be6ebca31ef01ca69965,2022-02-03 12:41:39,"[0, 0, 0, 13.623291025732051, 100, 0]",113.0,snapshot.dcl.eth,QmXxc9BLzcFvuVgFXEGoJwELpakEpjVS3W2spsamp4xzVd,0x5e23d08324f017d5425e59a2782c9ae27ace0958,2022-02-03 09:08:24,Should Yannakis be added to the Wearables Cura...,> by 0x76fb13f00cdbdd5eac8e2664cf14be791af87cb...,2022-02-03 09:08:00,2022-02-08 09:08:00,1
3,snapshot,Decentraland,c05d120d-4bb5-5b6a-89df-ed29b5336ec8,6e6af92a-ce8e-504f-8932-adc80f49eb82,41fd8de5-f8e2-5023-86a3-825c49e9ad7f,QmdC5dwN2K4q2AaTeAgiWcwrvfzJZHtqaQsw3mQ6nNJHhQ,0x0019220ddabe5b8438a6be6ebca31ef01ca69965,2022-02-16 21:09:40,"[0, 0, 0, 284.09829102573207, 100, 0]",384.0,snapshot.dcl.eth,Qmdyhx2tLVnFjiMkzQxTdzFZKA4ja1PqS8V5sVyQKGVdJf,0x5e23d08324f017d5425e59a2782c9ae27ace0958,2022-02-15 07:40:12,Ban the name childmolest,> by 0xe79574e9778c42fc2ddc33b38516e393e2cd80a...,2022-02-15 07:40:00,2022-02-22 07:40:00,1
4,snapshot,Decentraland,eb284263-38e1-57ed-adb1-101795d45882,146d86db-cf50-5e45-9c57-04af31b0600c,41fd8de5-f8e2-5023-86a3-825c49e9ad7f,0x9eacf88a6446989e9a2519be201ea36073eddfdaf81d...,0x0019220ddabe5b8438a6be6ebca31ef01ca69965,2022-11-19 13:36:46,"[0, 0, 0, 161.1956527363262, 100, 0, 0]",261.0,snapshot.dcl.eth,0x09764bf805351f645049fd6332da7be397e092dfb6f1...,0x5e23d08324f017d5425e59a2782c9ae27ace0958,2022-11-15 20:05:24,Vroomway Continuation - Race and Earn,> by 0x56469159d91eb810dce34dd13ec4ed8194bca7b...,2022-11-15 20:05:00,2022-11-29 20:05:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212253,NaN,NaN,NaN,2d7408c8-a32f-53d2-8125-8ea870d0574b,NaN,NaN,0xffe6d9d29480d0673c4fc46da22d21e245c8e731,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaT,0
212254,snapshot,Decentraland,cd86d88a-90b2-5976-8cca-346240ea5860,70241376-4762-52a3-83a8-1de78d04c139,41fd8de5-f8e2-5023-86a3-825c49e9ad7f,QmV3bybEyzeURWUnXkd1DvjYRqMeNPxvkiYzso5iKk1Qv3,0xffed2f64c9f796d4258618a7a375ed816205821a,2021-12-17 01:43:37,"[0, 0, 0, 1.37344292, 0, 0]",1.0,snapshot.dcl.eth,QmbNtCQbPG5sPLMnTdXLLURpqYSwvBM4P6vuWjwNX9Tn9s,0x5e23d08324f017d5425e59a2782c9ae27ace0958,2021-12-13 23:44:29,"Add the location -55,-129 to the Points of Int...",> by 0x38c02044322a1d0915f7905b35b89a635b261b4...,2021-12-13 23:44:00,2021-12-20 23:44:00,1
212255,NaN,NaN,NaN,3785a5a8-5834-59b5-a473-e5749063e217,NaN,NaN,0xffed2f64c9f796d4258618a7a375ed816205821a,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaT,0
212256,snapshot,Decentraland,b0827601-c9ca-576e-b797-a699ca1434ca,b6c6b750-0a5a-5ac0-9d2c-fe936957c1f4,41fd8de5-f8e2-5023-86a3-825c49e9ad7f,QmcLobabc6SdQM7gXXYzyKM7BfPNmYLRV

In [35]:
!tail /tmp/train.ffm

1 1:6860:1 2:7538:1
1 1:6860:1 2:8028:1
0 1:6860:1 2:7057:1
0 1:6860:1 2:7953:1
1 1:6861:1 2:7566:1
0 1:6861:1 2:7074:1
1 1:6862:1 2:8228:1
0 1:6862:1 2:7539:1
1 1:6863:1 2:8471:1
0 1:6863:1 2:8173:1


In [36]:
train_path = '/tmp/train.ffm'
test_path = '/tmp/test.ffm'

tr = pdu.negative_feedback_sampler(folds[0].train, col_user='voter', col_item='proposal', col_feedback='rating')
c.transform(tr).to_csv(train_path, sep=' ', index=False, header=False)
c.transform(folds[0].test.assign(rating=1)).to_csv(test_path, sep=' ', index=False, header=False)

In [37]:
model.fit(train_path, test_path)

step 1 , total_loss: nan, data_loss: nan


InvalidArgumentError: Graph execution error:

Detected at node 'XDeepFM/linear_part/linear_part/w' defined at (most recent call last):
    File "/home/daviddavo/.pyenv/versions/3.9.18/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    File "/home/daviddavo/.pyenv/versions/3.9.18/lib/python3.9/runpy.py", line 87, in _run_code
    File "/home/daviddavo/upm-tfm-notebooks/.direnv/python-3.9/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
    File "/home/daviddavo/upm-tfm-notebooks/.direnv/python-3.9/lib/python3.9/site-packages/traitlets/config/application.py", line 1053, in launch_instance
    File "/home/daviddavo/upm-tfm-notebooks/.direnv/python-3.9/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 736, in start
    File "/home/daviddavo/upm-tfm-notebooks/.direnv/python-3.9/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 195, in start
    File "/home/daviddavo/.pyenv/versions/3.9.18/lib/python3.9/asyncio/base_events.py", line 601